# Capstone Project

This notebook has been created to work on capstone project on Coursera.

In [20]:
import pandas as pd
import numpy as np

In [21]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## 1) Reading Neighborhood Data of London to Dataframe

First we will be fetching information on neighborhoods of London. We need data like name of neighborhood, the borough it belongs to and postal code for that area.

This information will give basic understading of neighborhoods on London.


In [22]:
#importing necessary libraries
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re

In [23]:
#reading table from wikipedia page into a list
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_areas_of_London")
table

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                 Download coordinates as: KML · GPX,
                                            Location  \
 0                                        Abbey Wood   
 1                                             Acton   
 2                                         Addington   
 3                                        Addiscombe   
 4                                       Albany Park   
 5                                  Aldborough Hatch   
 6                                           Aldgate   
 7                                           Aldwych   
 8                                          Alperton   
 9                                           Anerley   
 10                                            Angel   
 11                                        Aperfield   
 12                                          Archway   
 13                                   Ardleigh Green 

In [24]:
#reading from list into a dataframe
df = pd.DataFrame(table[1])
print(df.shape)
df.head()

(533, 6)


,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [25]:
#keeping only those neighborhood that belong to London
df = df[df['Post town']=='LONDON']
print(df.shape)
df.columns.values

(299, 6)


array(['Location', 'London\xa0borough', 'Post town',
       'Postcode\xa0district', 'Dial\xa0code', 'OS grid ref'],
      dtype=object)

In [26]:
#renaming columns
df = df[['Location','London\xa0borough','Postcode\xa0district']]
df.rename(columns={'London\xa0borough':'Borough','Postcode\xa0district':'Postcode'},inplace=True)
df.head()

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich [7]",SE2
1,Acton,"Ealing, Hammersmith and Fulham[8]","W3, W4"
6,Aldgate,City[10],EC3
7,Aldwych,Westminster[10],WC2
9,Anerley,Bromley[11],SE20


In [27]:
#cleaning Borough name to remove trailing numbers in bracket
b_list = []
for name in df['Borough']:
    temp = re.sub(r"\[\d+\]", "", name)
    b_list.append(temp.strip())
b_list
df['Borough'] = b_list
df.head()

,Location,Borough,Postcode
0,Abbey Wood,"Bexley, Greenwich",SE2
1,Acton,"Ealing, Hammersmith and Fulham","W3, W4"
6,Aldgate,City,EC3
7,Aldwych,Westminster,WC2
9,Anerley,Bromley,SE20


In [28]:
df['Postcode'].value_counts()

E14                    8
N1                     7
E1                     7
NW10                   7
SW1                    6
SE9                    5
EC1                    5
NW9                    5
W1                     5
E9                     4
E16                    4
SE12                   4
E11                    4
NW3                    4
WC2                    4
NW6                    4
W4                     4
NW1                    3
N4                     3
SW6                    3
E17                    3
E2                     3
NW2                    3
SW15                   3
SW19                   3
SE18                   3
N15, N17               3
N20                    3
WC1                    3
SE3                    3
N8                     3
SE6                    3
E3                     3
N15                    3
SE4                    3
SE1                    3
W12                    3
SE2                    3
N11                    3
SE13                   2



As we can see from above that there are multiple entries of Postal code in our dataset. 

Therefore we need to group by on Postal code and concat neighborhoods with a comma.


In [29]:
#doing group by to concat multiple neighborhood belonging to single Postcode
df['Neighborhood'] = df.groupby(['Postcode','Borough']).transform(lambda x: ','.join(x))
df1 = df.loc[:,['Postcode','Borough','Neighborhood']]

df1.drop_duplicates(inplace=True)
print(df1.shape)
df1.head(10)

(183, 3)


,Postcode,Borough,Neighborhood
0,SE2,"Bexley, Greenwich",Abbey Wood
1,"W3, W4","Ealing, Hammersmith and Fulham",Acton
6,EC3,City,Aldgate
7,WC2,Westminster,"Aldwych,Charing Cross,Covent Garden"
9,SE20,Bromley,"Anerley,Penge"
10,"EC1, N1",Islington,Angel
12,N19,Islington,"Archway,Upper Holloway"
15,"N11, N14",Enfield,Arnos Grove
16,SW12,Wandsworth,Balham
17,SE1,Southwark,"Bankside,Bermondsey"



## 2) Fetching coordinates of Neighborhoods from geonames.org

Now that we have the neighborhood data, we need their coordinates information as well.

This will help in getting nearby venue information.


In [30]:
#reading table into dataframe
tablep = pd.read_html("http://www.geonames.org/postalcode-search.html?q=london&country=GB")
tablep[2]
df_code = pd.DataFrame(tablep[2])
df_code.head()

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,London Bridge,SE1,United Kingdom,England,Greater London,NaN
1,NaN,51.499/-0.09,51.499/-0.09,51.499/-0.09,51.499/-0.09,51.499/-0.09,51.499/-0.09
2,2.0,Ealing,W5,United Kingdom,England,Greater London,NaN
3,NaN,51.512/-0.302,51.512/-0.302,51.512/-0.302,51.512/-0.302,51.512/-0.302,51.512/-0.302
4,3.0,Hackney,E8,United Kingdom,England,Greater London,NaN


In [31]:
#formatting dataframe
clist=[]
plist=[]
for i in range(0,len(df_code['Code'])):
    if (i%2)!=0:
        clist.append(df_code.loc[i,'Code'])
    else:
        plist.append(df_code.loc[i,'Code'])
df_ll = pd.DataFrame(zip(plist,clist),columns=['Postcode','Coordinates'])
df_ll.head()

,Postcode,Coordinates
0,SE1,51.499/-0.09
1,W5,51.512/-0.302
2,E8,51.55/-0.05
3,E9,51.539/-0.041
4,E13,51.527/0.012


In [32]:
#dividing coordinates into latitude and longitude
latlist=[]
lonlist=[]
for i in df_ll['Coordinates']:
    m = re.search('(.+)/(.+)', i)
    latlist.append(m.group(1))
    lonlist.append(m.group(2))
df_ll['Latitude'] = latlist
df_ll['Latitude']=df_ll['Latitude'].astype(float)
df_ll['Longitude'] = lonlist
df_ll['Longitude'] = df_ll['Longitude'].astype(float)
del df_ll['Coordinates']
df_ll = df_ll.groupby('Postcode').first()
df_ll.head()

,Latitude,Longitude
Postcode,,
AL2,51.725,-0.296
BR2,51.362,0.029
BR3,51.409,-0.025
BR4,51.367,-0.017
BR6,51.375,0.098


In [33]:
df_ll.dtypes

Latitude     float64
Longitude    float64
dtype: object

In [34]:
#joining with neighborhood dataframe to get final dataframe
df_final = pd.merge(df1,df_ll,on='Postcode',how='left')
print(df_final.shape)
df_final.head()

(183, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,SE2,"Bexley, Greenwich",Abbey Wood,51.487,0.107
1,"W3, W4","Ealing, Hammersmith and Fulham",Acton,NaN,NaN
2,EC3,City,Aldgate,51.509,-0.126
3,WC2,Westminster,"Aldwych,Charing Cross,Covent Garden",51.513,-0.123
4,SE20,Bromley,"Anerley,Penge",51.415,-0.050


In [35]:
#removing rows with nan values
df_final.dropna(inplace=True)
df_final = df_final[df_final['Neighborhood']!='Clapham']
print(df_final.shape)
df_final

(122, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,SE2,"Bexley, Greenwich",Abbey Wood,51.487,0.107
2,EC3,City,Aldgate,51.509,-0.126
3,WC2,Westminster,"Aldwych,Charing Cross,Covent Garden",51.513,-0.123
4,SE20,Bromley,"Anerley,Penge",51.415,-0.050
6,N19,Islington,"Archway,Upper Holloway",51.565,-0.135
8,SW12,Wandsworth,Balham,51.447,-0.138
9,SE1,Southwark,"Bankside,Bermondsey",51.499,-0.090
10,EC1,City,Barbican,51.524,-0.110
11,SW13,Richmond upon Thames,"Barnes,Castelnau",51.474,-0.248
13,SW11,Wandsworth,Battersea,51.475,-0.155



## 3) Vizualizing the Neighborhoods on Map

Now we are ready to visualize the neighborhoods of London on a map.

We will leverage folium library for this purpose.

This will help us to understand the proximity of the neighborhoods


In [36]:
!conda install -c conda-forge folium=0.5.0

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [37]:
#import necessary libraries
import folium

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [38]:
#generate a map to view neighborhoods
latitude = 51.509865   #london
longitude = -0.118092  #london

# create map of New York using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london


## 4) Fetching nearby venues for each Neighborhood

To begin our analysis we first need to fetch nearby venues for a given neighborhood.

For this we will leverage Foursquare location platform with our created account. 

This information will then be used for analysis and for recommending the best neighborhoods to open a Restaurant

In [39]:
#defining foursquare credentials
CLIENT_ID = 'LFK23QQLJ4AS4R1HANXPERTRLOKXSRUJ3HUADUY3M3IXYHH4' # your Foursquare ID
CLIENT_SECRET = '0M2D1BXDUM1IQIMIOQ2OUSNHA1MS2CX3ZZF4F50FXWPOFA5Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LFK23QQLJ4AS4R1HANXPERTRLOKXSRUJ3HUADUY3M3IXYHH4
CLIENT_SECRET:0M2D1BXDUM1IQIMIOQ2OUSNHA1MS2CX3ZZF4F50FXWPOFA5Q


In [40]:
#defining function to fetch nearby venues for a neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
#calling above function for each neighborhood
london_venues = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

Abbey Wood
Aldgate
Aldwych,Charing Cross,Covent Garden
Anerley,Penge
Archway,Upper Holloway
Balham
Bankside,Bermondsey
Barbican
Barnes,Castelnau
Battersea
Bayswater,Paddington
Belgravia,Knightsbridge,Millbank,Pimlico,St James's,Westminster
Bellingham,Catford,Southend
Belsize Park,Frognal,Hampstead,Swiss Cottage
Bethnal Green,Cambridge Heath
Blackfriars
Blackheath
Blackwall,Canary Wharf,Cubitt Town,Isle of Dogs,Leamouth,Limehouse,Millwall,Poplar
Bloomsbury,St Pancras
Bow,Bromley (also Bromley-by-Bow),Old Ford
Bowes Park,Wood Green
Brent Park,Church End,Harlesden,Stonebridge,Willesden
Brockley,Crofton Park,St Johns
Brondesbury,Queen's Park
Brunswick Park,Friern Barnet,New Southgate
Burroughs, The,Hendon
Camden Town,Chalk Farm,Somerstown
Cann Hall,Leytonstone
Canning Town,Custom House,North Woolwich,Silvertown
Charlton
Childs Hill
Chinatown,Marylebone (also St Marylebone),Mayfair,Soho
Chinbrook,Grove Park,Lee
Chingford,Highams Park
Church End
Clerkenwell,Finsbury,St Luke's
Colindale,Graha

In [42]:
print(london_venues.shape)
london_venues.head()

(4187, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbey Wood,51.487,0.107,Co-op Food,51.487650,0.113490,Grocery Store
1,Abbey Wood,51.487,0.107,Bostal Gardens,51.486670,0.110462,Playground
2,Abbey Wood,51.487,0.107,Roses Retail,51.486307,0.102429,Men's Store
3,Abbey Wood,51.487,0.107,Teddy Bears' Picnic-Greenwich & Bexley Communi...,51.483962,0.108919,Playground
4,Abbey Wood,51.487,0.107,Meghna Tandoori,51.485709,0.101681,Indian Restaurant



## 5) Analyzing Venues for each Neighborhood

Now that we have fetched information of nearby venues for all neighborhoods, let us start our analysis.

Let us first find the number of unique venue category for all neighborhood.

Then we will convert the category column to numeric using one hot encoding technique

We will then take a mean for each category for each neighborhood and sort it in descending order for each neighborhood. 

This will give us a fair understanding of the neighborhood characteristics.


In [43]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 265 uniques categories.


In [44]:
# one hot encoding to convert categorical data to numbers
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")
del london_onehot['Neighborhood']

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood']

#london_onehot.insert(0,'Neighborhood',london_venues['Neighborhood'])

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Beer Bar,Beer Store,Belgian Restaurant,Betting Shop,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,River,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Abbey Wood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [45]:
london_onehot.shape

(4187, 265)

In [46]:
#group by neighborhood as we need information at neighborhood level
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Beer Bar,Beer Store,Belgian Restaurant,Betting Shop,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,River,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Abbey Wood,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,

In [47]:
london_grouped.shape

(122, 265)

In [48]:
#defining function to fetch most common venues based on mean calculated above
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

We are interested to get top 10 most common venues for each neighborhood.


In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,Playground,Grocery Store,Men's Store,Indian Restaurant,Hotel,Duty-free Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
1,Aldgate,Theater,Hotel,Pub,Steakhouse,Ice Cream Shop,French Restaurant,Coffee Shop,Pizza Place,Wine Bar,Café
2,"Aldwych,Charing Cross,Covent Garden",Theater,Coffee Shop,Clothing Store,Bakery,Ice Cream Shop,Cosmetics Shop,Deli / Bodega,Dessert Shop,Sushi Restaurant,Tea Room
3,"Anerley,Penge",Pub,Pizza Place,Coffee Shop,Grocery Store,Supermarket,Garden Center,Fast Food Restaurant,Café,Train Station,Shopping Mall
4,"Archway,Upper Holloway",Coffee Shop,Grocery Store,Pub,Italian Restaurant,Café,Bar,Pizza Place,Indian Restaurant,Fast Food Restaurant,Gym / Fitness Center


In [50]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,Playground,Grocery Store,Men's Store,Indian Restaurant,Hotel,Duty-free Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
1,Aldgate,Theater,Hotel,Pub,Steakhouse,Ice Cream Shop,French Restaurant,Coffee Shop,Pizza Place,Wine Bar,Café
2,"Aldwych,Charing Cross,Covent Garden",Theater,Coffee Shop,Clothing Store,Bakery,Ice Cream Shop,Cosmetics Shop,Deli / Bodega,Dessert Shop,Sushi Restaurant,Tea Room
3,"Anerley,Penge",Pub,Pizza Place,Coffee Shop,Grocery Store,Supermarket,Garden Center,Fast Food Restaurant,Café,Train Station,Shopping Mall
4,"Archway,Upper Holloway",Coffee Shop,Grocery Store,Pub,Italian Restaurant,Café,Bar,Pizza Place,Indian Restaurant,Fast Food Restaurant,Gym / Fitness Center
5,Balham,Tennis Court,Park,Zoo Exhibit,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Film Studio,Electronics Store
6,"Bankside,Bermondsey",Pub,Italian Restaurant,Garden,Park,Café,Deli / Bodega,Shoe Store,Hostel,Fast Food Restaurant,Whisky Bar
7,Barbican,Pub,Coffee Shop,Hotel,Café,Pizza Place,Sandwich Place,Bar,Park,Vietnamese Restaurant,Italian Restaurant
8,"Barnes,Castelnau",Pub,Farmers Market,Pizza Place,Grocery Store,Café,Gym / Fitness Center,French Restaurant,Lake,Athletics & Sports,Train Station
9,Battersea,Bar,Chinese Restaurant,Soccer Field,Italian Restaurant,Pub,Furniture / Home Store,Gymnastics Gym,Art Gallery,Coffee Shop,Flea Market


Now the best location to open a Restaurant would be areas with higher footprint, i.e. areas which are busy and hence the restaurant is visible to public. For this we choose location which have transportation hub nearby or shopping malls nearby. These are the most busiest places and with best prospectus for opening a busineess specially restaurant for people to relax and have food.

In [51]:
#defining a list for transportation hub and shopping malls
implist = ['Bus ','Train ','Metro ','Airport ','Shopping ']
categories = london_venues['Venue Category'].unique()
yeslist = []
for category in categories:
    for word in implist:
        if word in category:
            yeslist.append(category)
yeslist

['Shopping Plaza',
 'Shopping Mall',
 'Train Station',
 'Bus Stop',
 'Metro Station',
 'Bus Station',
 'Airport Service',
 'Airport Terminal']

Also the area should not have too much competition to start with. In our case the area should not have too many reataurants already as this will hamper the number of customers and hence the revenue. 

In [52]:
#defining a list with categories having restaurant in their names
nolist = []
for category in categories:
    if 'Restaurant' in category:
        nolist.append(category)
nolist

['Indian Restaurant',
 'Spanish Restaurant',
 'Mexican Restaurant',
 'Italian Restaurant',
 'North Indian Restaurant',
 'Seafood Restaurant',
 'Portuguese Restaurant',
 'Persian Restaurant',
 'Sushi Restaurant',
 'Asian Restaurant',
 'African Restaurant',
 'French Restaurant',
 'English Restaurant',
 'Restaurant',
 'Peruvian Restaurant',
 'Japanese Curry Restaurant',
 'Japanese Restaurant',
 'Thai Restaurant',
 'Tapas Restaurant',
 'Korean Restaurant',
 'Lebanese Restaurant',
 'Mediterranean Restaurant',
 'Belgian Restaurant',
 'Fast Food Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Kebab Restaurant',
 'Middle Eastern Restaurant',
 'Vietnamese Restaurant',
 'Falafel Restaurant',
 'Modern European Restaurant',
 'Greek Restaurant',
 'Chinese Restaurant',
 'Dim Sum Restaurant',
 'Brazilian Restaurant',
 'Malay Restaurant',
 'Moroccan Restaurant',
 'Turkish Restaurant',
 'Argentinian Restaurant',
 'American Restaurant',
 'Arepa Restaurant',
 'Himalayan Restaurant',
 'Afghan Restaurant'

## 6) Final Result

Now finally let us select the cluster of Neighborhood which have either transportation hub nearby or shopping mall nearby. Also the cluster should filter neighborhoods which already have many reataurants present.

In [53]:
#applying conditions to select the best cluster of neighborhoods with satisfy our requirements
neighborhoods_venues_sorted[(neighborhoods_venues_sorted['1st Most Common Venue'].isin(yeslist)\
                            | neighborhoods_venues_sorted['2nd Most Common Venue'].isin(yeslist)\
                           | neighborhoods_venues_sorted['3rd Most Common Venue'].isin(yeslist))\
                           & ~(neighborhoods_venues_sorted['1st Most Common Venue'].isin(nolist)\
                            | neighborhoods_venues_sorted['2nd Most Common Venue'].isin(nolist)\
                           | neighborhoods_venues_sorted['3rd Most Common Venue'].isin(nolist))]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,"Brunswick Park,Friern Barnet,New Southgate",Bus Stop,Park,Beer Bar,Metro Station,Pool,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio
28,"Canning Town,Custom House,North Woolwich,Silve...",Airport Service,Sandwich Place,Duty-free Shop,Italian Restaurant,Theater,Breakfast Spot,Coffee Shop,Airport Terminal,Hotel,Currency Exchange
29,Charlton,Bus Stop,Grocery Store,IT Services,Thai Restaurant,Pet Store,Construction & Landscaping,Platform,Pub,Chinese Restaurant,Café
53,Forest Gate,Grocery Store,Train Station,Pub,Bakery,Café,Fast Food Restaurant,Flower Shop,Fish & Chips Shop,Film Studio,Flea Market
102,"Seven Sisters,Tottenham Green,West Green",Coffee Shop,Pub,Bus Stop,Café,Train Station,Grocery Store,Fried Chicken Joint,Tapas Restaurant,Market,South American Restaurant


## The above output provides information on 5 best neighborhood to open a Restaurant.

### Thank You!